In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import time
from datetime import datetime
pd.set_option("display.max_colwidth",50)

In [2]:
Start = "https://www.presseportal.de/blaulicht/nr/4970"
html = urlopen(Start)
bs = BeautifulSoup(html, 'html.parser')
Links=bs.find_all("h2", class_="news-headline-clamp" )

In [3]:
#Try Pages not found again
def getPagesNotfound(AllPagesNotfound):
    
    LinksNotFoundBefore=[]
    LinksStillNotFOund=[]
    for SinglePage in AllPagesNotfound:
        try:
            bs = BeautifulSoup(SinglePage, 'html.parser')
            Tag=bs.find(class_="pagination-next")
            
            LinksNotFoundBefore.append("https://www.presseportal.de"+Tag.attrs['data-url'][1:])
            time.sleep(.3)
        except:
            time.sleep(1)
            LinksStillNotFOund.append("https://www.presseportal.de"+Tag.attrs['data-url'][1:])
    return LinksNotFoundBefore, LinksStillNotFOund
 
    
#getallsites
Start = "https://www.presseportal.de/blaulicht/nr/4970"
def getallsites(Start, MaxP):
    Pages=[]
    PagesNotfound=[]
    html = urlopen(Start)
    Success =0
    NoSuccess=0
    for i in range(1,MaxP+1):
        try:
            bs = BeautifulSoup(html, 'html.parser')
            Tag=bs.find(class_="pagination-next")
            Pages.append("https://www.presseportal.de"+Tag.attrs['data-url'][1:])
            html = urlopen("https://www.presseportal.de"+Tag.attrs['data-url'][1:])
            time.sleep(.3)
            Success +=1
        except:
            time.sleep(1)
            NoSuccess +=1
            PagesNotfound.append("https://www.presseportal.de"+Tag.attrs['data-url'][1:])
        time.sleep(.3)
        print("getallsites Progress "+str(round(((i/MaxP)*100),2))+"%" + " Success: " +str(Success) + " NoSuccess: "+str(NoSuccess), flush=True,end='\r')
    LinksNotFoundBefore,LinksStillNotFOund = getPagesNotfound(PagesNotfound)
    
    for Link in LinksNotFoundBefore:
        Pages.append(Link)
    
    return Pages,LinksStillNotFOund


In [4]:
AllSites, SitesNotFound =getallsites(Start,20)

In [5]:
def getCorpusFromLink(AllSites):
    MainsiteCorpus=[]
    RemainingSites= []
    for Site in AllSites:
        try:
            html = urlopen(Site)
            bs = BeautifulSoup(html, 'html.parser')
            Links=bs.find_all("h3", class_="news-headline-clamp" )
            MainsiteCorpus.append(Links)
        except:
            RemainingSites.append(Site)
            
    return MainsiteCorpus, RemainingSites

def getLinksFromCorpus(MainsiteCorpus):
    Linklist =[]
    for Site in MainsiteCorpus:
        for Headline in Site:
            Article= (Headline.findChild("")["href"])
            Linklist.append(Article)
    return Linklist

def findMainArticleForGivenText(Corpus):
    s = ''
    for element in Corpus:
        s += '\n' + ''.join(element.findAll(text = True))
        s
    start = 'Polizeipräsidium Frankfurt am Main'
    end = 'Rückfragen bitte an:'
    s = (s[s.find(start)+len(start):s.rfind(end)])
    return s

In [6]:
def getNewsFromLinks(Liste):
    counter=0
    counter2=0
    News=[]
    Headline=[]
    Hauptartikel=[]
    LinkNotfound=[]
    for Link in Liste:
        try:
            html=urlopen('https://www.presseportal.de{}'.format(Link))
            bs = BeautifulSoup(html, 'html.parser')
            Ueberschrift= bs.find("h1").get_text()
            WholeText =bs.find_all('p')
            MainArticle =findMainArticleForGivenText(WholeText)
            Headline.append(Ueberschrift)
            Hauptartikel.append(MainArticle)
            counter+=1
            News.append('https://www.presseportal.de{}'.format(Link))
            time.sleep(.3)
            print("found-> "+ str(counter) + "/"+ str(counter2)+ " of "+ str(len(Liste)),flush=True,end='\r')
        except:
            counter2+=1
            LinkNotfound.append(Link)
            print("found-> "+ str(counter) + "/"+ str(counter2)+ " of "+ str(len(Liste)),flush=True,end='\r')
        
    return Headline,Hauptartikel,News,LinkNotfound

In [7]:
Corpus, Other =getCorpusFromLink(AllSites)

In [9]:
LinksForAllNews = getLinksFromCorpus(Corpus)

In [10]:
#AlleHeadlines, AlleHauptartikel, AlleWebsiten, LinkNotfound =getNewsFromLinks(LinksForAllNews)

In [49]:
df = pd.DataFrame({'Headline':AlleHeadlines,'Hauptartikel':AlleHauptartikel,'Link':AlleWebsiten})

In [50]:
df.head()

,Headline,Hauptartikel,Link
0,POL-F: 190418 - 444 Frankfurt-Griesheim: Unfal...,\n\nFrankfurt (ots)\n (ka) Heute Morgen stieße...,https://www.presseportal.de/blaulicht/pm/4970/...
1,POL-F: 190418 - 443 Frankfurt-Stadtgebiet: Ges...,\n\nFrankfurt (ots)\n (fue) Auch in der kommen...,https://www.presseportal.de/blaulicht/pm/4970/...
2,POL-F: 190418 - 442 Frankfurt-Oberrad: Wohnung...,\n\nFrankfurt (ots)\n (fue) Die 68-jährige Mie...,https://www.presseportal.de/blaulicht/pm/4970/...
3,POL-F: 190418 - 441 Bundesautobahn 5: Zeugensuche,"\n\nFrankfurt (ots)\n (fue) Am Mittwoch, den 1...",https://www.presseportal.de/blaulicht/pm/4970/...
4,POL-F: 190417 - 440 Frankfurt-Innenstadt: Fest...,\n\nFrankfurt (ots)\n (em) Gestern (16.04.2019...,https://www.presseportal.de/blaulicht/pm/4970/...


In [51]:
#Replace escape sharacters
df['Headline']= df["Headline"].apply(lambda st: st.replace("\t"," "))
df['Headline']= df["Headline"].apply(lambda st: st.replace("\n"," "))
df['Hauptartikel']= df["Hauptartikel"].apply(lambda st: st.replace("\n"," "))

df["date"] =df.Headline.str.extract('(\d+)')
df['date'] = df['date'].apply(lambda x: "{}{}".format('20', x))
#ffill if date is not found
df['date'] = df['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d') if len(x)==8 else "").fillna(method='ffill')
df['Headline'] = df['Headline'].str.replace('\d+', '')

In [52]:
#Extract case
#df["case"] =df.Headline.str.extract('(\s\d{3,4}\s)')
#df["case"] = df["case"].astype(str)

In [53]:
#Find index of Case in Headline
#df['col3'] = [i2.index(i1) for i1,i2 in zip(df.case,df.Headline)]
#COnvert into numeric
#df['col3'] = pd.to_numeric(df['col3'] )
#Remove beginning of Headline
#df['Headline'] = [A[B:] for A, B in zip(df["Headline"], df["col3"])]

In [54]:
#Strip whitespaces
df['Headline']= df["Headline"].apply(lambda st: st.lstrip())
#Remove beginning from Headline (remove 4digit Number)
df['Headline']=df['Headline'].str.replace("\d{4}","",regex=True)
df['Headline']=df['Headline'].str.replace("\d{3}","",regex=True)
df['Headline']=df['Headline'].str.replace("POL-F:","")

#Strip whitespaces again
df['Headline']= df["Headline"].apply(lambda st: st.lstrip())

In [55]:
#Function to find nth substring
def find_nth(s, substr, n):
    i = 0
    while n >= 0:
        n -= 1
        i = s.find(substr, i + 1)
    return i

In [56]:
#Extract Location
df['Location']= df["Headline"].apply(lambda st: st[0:st.find(":")])
#remove Stuff from Location
df['Location']=df['Location'].str.replace("/"," ")
df['Location']=df['Location'].str.replace("Frankfurt","")
df['Location']=df['Location'].str.replace("-"," ")
df['Location']= df["Location"].apply(lambda st: st.lstrip())
#Remove Headline
df['Headline']= df["Headline"].apply(lambda st: st[st.find(":")+1:])
#Strip whitespace again
df['Headline']= df["Headline"].apply(lambda st: st.lstrip())
#Extract second location
df['Location2']= df['Location'].str.split('/').str[1]
#Clean Location
df['Location']= df['Location'].str.split('/').str[0]
#Remove start of Hauptartikel e.g. Frankfurt (ots) - (ka)
df["Headline"].apply(lambda st: st[st.find(" - ")+2:find_nth(st," ",3)])
#Extract start of Hauptartikel
df["Beschreibung"]= df["Hauptartikel"].apply(lambda st: st[0:find_nth(st,")",1)+1])
#Remove start from Hauptartikel
df["Hauptartikel"]= df["Hauptartikel"].apply(lambda st: st[find_nth(st,")",1)+1:])

In [57]:
df=df.drop_duplicates()
df.count()

Headline        539
Hauptartikel    539
Link            539
date            539
Location        539
Location2         0
Beschreibung    539
dtype: int64

In [58]:
df.to_csv("Test.csv",sep="@",encoding="UTF-16")

In [59]:
df

,Headline,Hauptartikel,Link,date,Location,Location2,Beschreibung
0,Unfall zwischen Auto und Straßenbahn,Heute Morgen stießen auf der Mainzer Landstra...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-18,Griesheim,NaN,Frankfurt (ots) (ka)
1,Geschwindigkeitsmessungen,Auch in der kommenden Woche werden im Frankfu...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-18,Stadtgebiet,NaN,Frankfurt (ots) (fue)
2,Wohnungsbrand,Die 68-jährige Mieterin einer Wohnung in der ...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-18,Oberrad,NaN,Frankfurt (ots) (fue)
3,Zeugensuche,"Am Mittwoch, den 17. April 2019, gegen 22.10 ...",https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-18,Bundesautobahn,NaN,Frankfurt (ots) (fue)
4,Festnahme eines Jugendlichen,Gestern (16.04.2019) hat ein 14-Jähriger glei...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-17,Innenstadt,NaN,Frankfurt (ots) (em)
5,Lastwagen umgekippt,Gestern Nachmittag fiel ein Lastwagen auf der...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-17,Bundesautobahn,NaN,Frankfurt (ots) (ka)
6,Frau an Straßenbahnhaltestelle verletzt,Eine 77-jährige Frankfurterin befand sich am ...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-17,Bornheim,NaN,Frankfurt (ots) (fue)
7,Zeugensuche!,Nach dem bisherigen Ermittlungsstand gerieten...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-17,Nordend,NaN,Frankfurt (ots) (fue)
8,Schwerer Verkehrsunfall,"Heute, am 16. April 2019, gegen 12.05 Uhr, wu...",https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-16,Bundesautobahn,NaN,Frankfurt (ots) (fue)
9,Verkehrsunfall,Ein 61-jähriger Mann aus Nidderau befuhr am M...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-16,Innenstadt,NaN,Frankfurt (ots) (fue)


In [87]:
#df["Clean_Hauptartikel"] = df['Hauptartikel'].str.replace('[^\w\s]','')

,Headline,Hauptartikel,Link,date,case,col3,Clean_Hauptartikel
0,Frankfurt-Sossenheim: Auto geklaut - Polizei e...,Frankfurt (ots) (pe) Dienstagabend (09.04.2...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-11,413,15,Frankfurt ots pe Dienstagabend 09042019 mel...
1,Frankfurt-Bornheim: Trickdiebin bringt Seniori...,Frankfurt (ots) (pe) Am Dienstag (09.04.201...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-11,412,15,Frankfurt ots pe Am Dienstag 09042019 haben...
2,Frankfurt-Eschersheim: Frau schlägt Räuber in ...,Frankfurt (ots) (pe) In der vergangenen Nac...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-11,411,15,Frankfurt ots pe In der vergangenen Nacht 1...
3,Frankfurt-BAB3: Schwerer Verkehrsunfall mit ei...,Frankfurt (ots) (em) Heute Vormittag (10.04...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-10,410,15,Frankfurt ots em Heute Vormittag 10042019 k...
4,Frankfurt-Praunheim: Rennradfahrer fährt Fußg...,Frankfurt (ots) (pe) Am gestrigen Mittag (0...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-10,409,15,Frankfurt ots pe Am gestrigen Mittag 090420...
5,Frankfurt: Fälle sexueller Belästigungen - Pol...,Frankfurt (ots) (em) Wie den Medien bereits...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-10,408,15,Frankfurt ots em Wie den Medien bereits bek...
6,Frankfurt-Bahnhofsviertel: Parfumplagiat im We...,Frankfurt (ots) (pe) Am gestrigen Abend (09...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-10,407,15,Frankfurt ots pe Am gestrigen Abend 0904201...
7,Frankfurt-Bahnhofsviertel: Autodieb ermittelt ...,Frankfurt (ots) (pe) In der Nacht von Diens...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-10,406,15,Frankfurt ots pe In der Nacht von Dienstag ...
8,Frankfurt-Innenstadt: 31-Jähriger baut Unfall ...,Frankfurt (ots) (ka) In der Nacht von Sonnt...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-09,405,15,Frankfurt ots ka In der Nacht von Sonntag a...
9,Frankfurt: Einladung zur Präventionsveranstalt...,Frankfurt (ots) (em) Wer kennt es nicht? Sc...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-09,404,15,Frankfurt ots em Wer kennt es nicht Schnell...
